In [1]:
from forecasting_tools.data_models.questions import MetaculusQuestion
from forecasting_tools.helpers.metaculus_client import MetaculusClient, ApiFilter
from forecasting_tools.helpers.metaculus_api import MetaculusApi

AIB_FALL_2025_ID = 32813 
metaculus_client = MetaculusClient(token="c847890e6bf253ff0d3426be08fc3eb61f87606c")
print(metaculus_client.token)
# metaculus_api = MetaculusApi()
import logging
logging.basicConfig(level=logging.INFO)
import asyncio

api_filter = ApiFilter(
            allowed_statuses=["open"],
            allowed_types=["binary", "numeric"],
            num_forecasters_gte=12,
            includes_bots_in_aggregates=False,
            community_prediction_exists=True,
        )

questions = []
for offset in range(10):
    print(offset, len(questions))
    url_params = metaculus_client._create_url_params_for_search(api_filter, offset*100)
    # print(url_params)
    questions_update = metaculus_client._get_questions_from_api(
                url_params, api_filter.group_question_mode
            )
    questions.extend(questions_update)

2026-02-13 09:45:21.548 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


c847890e6bf253ff0d3426be08fc3eb61f87606c
0 0
1 100
2 200
3 300
4 400
5 500
6 600
7 700
8 800
9 900


In [22]:
import pendulum
from datetime import datetime, timedelta

curr_date = pendulum.now(tz="UTC")
date_into_past = pendulum.now(tz="UTC") - timedelta(days=50)
print(curr_date > date_into_past)

True


In [43]:
questions[0].scheduled_resolution_time

DateTime(2027, 1, 1, 5, 0, 0, tzinfo=Timezone('UTC'))

In [55]:
high_forecast_questions = [q for q in questions if (q.num_forecasters > 15
                            and q.cp_reveal_time <= curr_date 
                            and q.open_time >= date_into_past
                            and q.close_time > curr_date
                            and q.scheduled_resolution_time > curr_date + timedelta(days=15)
                            and q.scheduled_resolution_time < curr_date + timedelta(days=35))
                             ]
print(len(high_forecast_questions))

18


In [56]:
for q in high_forecast_questions:
    print(q.question_text, q.scheduled_resolution_time)
    if q.question_type == 'numeric':
        print(q.unit_of_measure, q.lower_bound, q.upper_bound)

Will the SWE-bench Bash Only benchmark have a new high score before March 14, 2026? 2026-03-14 00:00:00+00:00
What will be the average weekly YoY growth of the Redbook Same-Store Retail Sales Index during February 2026? 2026-03-04 00:00:00+00:00
% 4.0 9.0
How much higher will the Shanghai–LA spot container freight rate be than the Shanghai–Rotterdam rate on March 5, 2026? 2026-03-06 00:00:00+00:00
$/40ft -1000.0 2000.0
What will be the U.S. Dollar to Argentine Peso exchange rate on March 2, 2026? 2026-03-02 00:00:00+00:00
pesos 1350.0 1600.0
What will be the average live TV viewership rating of China Media Group's 2026 Chinese New Year’s Gala? 2026-03-14 06:00:00+00:00
% 20.0 45.0
Will the U.S. Congress still be operating under a continuing resolution (CR) on March 15, 2026? 2026-03-15 00:00:00+00:00
Will OpenAI's ChatGPT Atlas browser be released for Windows before March 14, 2026? 2026-03-14 05:00:00+00:00
In February 2026, will the weekly amount of finished motor gasoline supplied in

In [57]:
qid = [q.id_of_post for q in high_forecast_questions]
print(qid)

[41806, 41683, 41586, 41583, 41528, 41512, 41500, 41361, 41358, 41347, 41343, 41341, 41337, 41336, 41316, 41302, 40726, 40713]


In [4]:
from collections import Counter
type_counts = Counter([q.question_type for q in questions])
print(type_counts)

Counter({'binary': 772, 'numeric': 228})


In [2]:
import asyncio

api_filter = ApiFilter(
            allowed_statuses=["open"],
            allowed_types=["binary", "numeric"],
            num_forecasters_gte=12,
            includes_bots_in_aggregates=False,
            community_prediction_exists=True,
        )

# questions = asyncio.run(
#     metaculus_client._filter_sequential_strategy(api_filter, 1)
# )

NameError: name 'ApiFilter' is not defined

In [27]:
url_params = metaculus_client._create_url_params_for_search(api_filter, 0)
print(url_params)
questions = metaculus_client._get_questions_from_api(
            url_params, api_filter.group_question_mode
        )

{'limit': 100, 'offset': 0, 'order_by': '-published_time', 'with_cp': 'true', 'include_conditional_cps': 'true', 'forecast_type': ['binary', 'numeric'], 'statuses': ['open']}


In [51]:
metaculus_client.get_question_by_post_id(41953)

INFO:forecasting_tools.helpers.metaculus_client:Retrieving question details for question 41953
INFO:forecasting_tools.helpers.metaculus_client:Retrieved question details for question 41953


NumericQuestion(question_text='What will be the highest price of silver per troy ounce in April 2026?', id_of_post=41953, page_url='https://www.metaculus.com/questions/41953', id_of_question=41692, state=<QuestionState.OPEN: 'open'>, num_forecasters=79, num_predictions=396, resolution_criteria='This question will resolve as the highest price of silver (SI=F) displayed by [Yahoo Finance](https://finance.yahoo.com/quote/SI%3DF/history/) for any day in April 2026.', fine_print='Specifically, the highest value shown in the "High" column will be used to resolve this question. For example, on January 30, 2026, the value shown was \\$117.79.', background_info='After opening 2025 at \\$30 per troy ounce, silver went on a bull run in 2025, with the white metal gaining more than 279% at its highest point. Prior to October 9, 2025, silver\'s all-time high had been \\$49.95, set forty-five years prior, on January 17, 1980. At the end of 2025, [experts continued to be optimistic for the future of s

In [1]:
import os

post_id = 41953
token = os.environ["METACULUS_TOKEN"]  # set this in your env/.env
print(token)
out = f"post_{post_id}_data.zip"

!curl -f -L \
  -H "Accept: application/zip" \
  -H "Authorization: Token {token}" \
  "https://www.metaculus.com/api/posts/{post_id}/download-data/" \
  -o "{out}"

print("saved:", out)


a44cf1d8ecd4d570895e736123d9605d99e1f6ee
saved: post_41953_data.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (22) The requested URL returned error: 403


In [47]:
print(questions[10].already_forecasted)

False


In [ ]:
print(

AttributeError: 'NumericQuestion' object has no attribute 'keys'

In [37]:
for i, question in enumerate(questions):
    print(i, question.cp_reveal_time, question.num_forecasters)

0 2026-02-15 19:39:23+00:00 1
1 2026-02-14 15:39:10+00:00 10
2 2026-02-15 13:00:00+00:00 4
3 2026-02-12 08:26:20+00:00 8
4 2026-02-17 17:00:00+00:00 40
5 2026-02-12 10:09:00+00:00 1
6 2026-02-12 10:08:00+00:00 1
7 2026-02-12 10:08:00+00:00 2
8 2026-02-12 10:08:00+00:00 1
9 2026-02-08 01:20:42+00:00 5
10 2026-02-07 16:00:00+00:00 79
11 2026-02-01 22:32:00+00:00 46
12 2025-08-21 12:00:00+00:00 0
13 2026-02-06 03:47:22+00:00 15
14 2026-02-16 17:00:00+00:00 38
15 2026-02-08 02:27:39+00:00 17
16 2026-02-10 00:36:25+00:00 12
17 2026-02-04 03:05:07+00:00 8
18 2026-02-09 16:00:00+00:00 438
19 2026-02-03 13:53:25+00:00 14
20 2026-02-07 04:00:00+00:00 1
21 2026-02-23 16:00:00+00:00 435
22 2026-02-16 16:00:00+00:00 393
23 2026-02-16 16:00:00+00:00 386
24 2026-02-04 13:45:24+00:00 23
25 2026-02-09 01:54:13+00:00 10
26 2026-02-02 16:00:00+00:00 528
27 2026-01-30 19:19:41+00:00 12
28 2026-02-06 03:05:31+00:00 6
29 2026-02-16 16:00:00+00:00 525
30 2026-02-04 01:41:30+00:00 7
31 2026-02-09 16:00:00+00

In [28]:
for question in questions:
    print(question.num_forecasters, question.question_type)

1 binary
10 binary
4 numeric
8 binary
40 numeric
1 binary
1 binary
2 binary
1 binary
5 binary
79 numeric
46 numeric
0 binary
15 binary
38 numeric
17 binary
12 binary
8 binary
438 binary
14 binary
1 binary
435 binary
393 numeric
386 numeric
23 binary
10 binary
528 numeric
12 binary
6 binary
525 numeric
7 numeric
553 numeric
77 binary
221 binary
523 numeric
524 numeric
560 numeric
29 binary
31 binary
24 binary
11 binary
5 numeric
28 binary
706 numeric
38 binary
28 binary
15 binary
135 binary
28 binary
148 binary
142 binary
807 binary
734 binary
13 binary
23 binary
36 binary
316 binary
14 binary
5 binary
82 binary
7 binary
18 binary
12 binary
718 binary
9 numeric
731 binary
180 numeric
1430 binary
780 numeric
25 binary
147 binary
1356 binary
1498 binary
1362 binary
404 numeric
1146 numeric
40 binary
767 binary
33 binary
26 binary
29 binary
773 binary
29 binary
5 numeric
11 binary
685 binary
781 binary
726 binary
738 binary
11 binary
4 numeric
5 binary
5 binary
5 numeric
8 binary
5 binary


In [ ]:
print(matches)

[]


In [5]:
benchmark_questions = metaculus_client.get_benchmark_questions(2)

INFO:forecasting_tools.helpers.metaculus_client:Retrieving 2 benchmark questions
INFO:forecasting_tools.helpers.metaculus_client:Estimating that there are 300 questions matching the filter -> num_forecasters_gte=30 allowed_types=['binary'] allowed_subquestion_types=None group_question_mode='exclude' allowed_statuses=['open'] scheduled_resolve_time_gt=None scheduled_resolve_time_lt=None pub
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<MetaculusClient.get_questions_matching_filter() done, defined at c:\Users\amdic\miniconda3\envs\calp\Lib\site-packages\forecasting_tools\helpers\metaculus_client.py:374> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "c:\Users\amdic\miniconda3\envs\calp\Lib\site-packages\IPython\core\interactiveshell.py", line 3701, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\amdic\AppData\Local\Temp\ipykernel_10496\1613020863.py", line 11, in <module>
    benchm

ValueError: Exhausted all 3 pages but only found 0 questions, needed 2. Set error_if_not_enough_questions to False to return as many as possible